## 뉴럴 네트워크는 torch.nn 패키지를 이용하여 생성할 수 있다.

일반적인 Neural Networks의 학습 절차는 다음과 같다.

- 학습 가능한 파라미터나 weight가 있는 뉴럴 네트워크를 정의한다.
- 입력 데이터셋에 대한 반복 학습을 진행한다.
- 네트워크를 통해 입력 값을 처리한다.
- loss를 계산한다
- 손실 계산을 한다. (손실 = 출력값(output)과 정답(target)의 차이 
- gradient를 네트워크의 파라미터로 역전파(backward) 시킨다.
- 다음과 같이 간단한 식을 통해 네트워크의 weight를 갱신한다.
    - weight = weight-learning_rate * gradient

## 신경망 정의

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        # in_ch=1, out_ch=6, kenerl_size=5 
        self.conv2 = nn.Conv2d(6,16,5)
        # in_ch=6, out_ch=16, kenerl_size=5
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


사용자는 직접 forward 함수만 정의해주면 된다. Gradient가 계산되는 backward 함수는 autograd를 사용함으로써 자동으로 정의 된다. 사용자는 forward함수에서 모든 텐서 연산을 사용할 수 있다.

In [11]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [12]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0682, -0.0400,  0.0990, -0.0406,  0.0156, -0.0649,  0.0696, -0.0634,
          0.1471, -0.0527]], grad_fn=<AddmmBackward>)


In [13]:
net.zero_grad()
out.backward(torch.randn(1,10))

- torch.Tensor : backward()와 같은 autograd연사을 지원하는 다차원 배열이며 텐서에 대한 gradient(기울기)를 가지고 있다.
- nn.Module : 뉴럴 네트워크 모듈로서 파라미터를 GPU로 옮기거나, 내보내기, 불러오기 등의 보조 작업을 이용하여 파라미터를 캡슐화 하는 편리한 방법
- nn.Parameter : 텐서의 일종으로 Module에 속성을 할당 될 때 파라미터로 자동 등록된다.
- autograd.Function - autograd 연산의 전방향과 역방향 정의 를 구현합니다. 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor 를 생성하고 이력(history)을 부호화 하는 함수들과 연결하고 있습니다.


## 손실함수(Loss Function)

In [14]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5025, grad_fn=<MseLossBackward>)


## 역전파(Backpropagation)

In [15]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0016,  0.0032, -0.0075,  0.0019,  0.0034,  0.0054])


## Weight 갱신

In [17]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

In [19]:
import torch.optim as optim

# Optimizer 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

nn을 사용할 때 SGD, Adam 등 과 같은 다양한 갱신 룰을 사용하고 싶을 수 있다. 이러한 룰의 사용을 위해 PyTorch는 위와 같은 갱신 룰을 포함하는 torch.optim 패키지를 제공한다